In [ ]:
# ===============================
# SMART GRID STABILITY ANALYSIS PIPELINE
# ===============================
# Author: Research Workflow for Smart Grid Fault Detection (MLP-HF)
# ===============================

# --- IMPORT LIBRARIES ---
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             matthews_corrcoef, roc_curve, roc_auc_score, confusion_matrix)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy import stats
import shap
import lime
import lime.lime_tabular
import time
import warnings
warnings.filterwarnings("ignore")

# --- LOAD DATASET ---
df = pd.read_csv("/content/drive/MyDrive/dataset.csv")

# --- PREPROCESSING ---
df = df.drop_duplicates().dropna()
le = LabelEncoder()
df['stabf'] = le.fit_transform(df['stabf'])  # 1 = unstable, 0 = stable

X = df.drop(['stab', 'stabf'], axis=1)
y = df['stabf']

# Normalization and Standardization
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# --- FEATURE SELECTION METHODS ---

## 1. PCA Feature Reduction
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_scaled)
explained_var = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(8,5))
plt.plot(range(1, len(explained_var)+1), explained_var, marker='o', color='b')
plt.title("Explained Variance (PCA)", fontsize=14)
plt.xlabel("Principal Components")
plt.ylabel("Cumulative Explained Variance")
plt.grid()
plt.show()

## 2. Mutual Information
mi = mutual_info_classif(X_scaled, y)
mi_scores = pd.Series(mi, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(x=mi_scores, y=mi_scores.index, palette="viridis")
plt.title("Feature Importance via Mutual Information")
plt.xlabel("Mutual Information Score")
plt.show()

## 3. Correlation Matrix
corr = X_scaled.corrwith(y)
plt.figure(figsize=(8,5))
corr.sort_values().plot(kind='barh', color='teal')
plt.title("Feature Correlation with Target")
plt.show()

## 4. KMeans Clustering with Silhouette Analysis
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(X_scaled)
df['Cluster'] = clusters
sil = silhouette_score(X_scaled, clusters)
print(f"Silhouette Score: {sil:.3f}")

plt.figure(figsize=(8,5))
sns.heatmap(X_scaled.corr(), cmap='coolwarm', center=0)
plt.title("Feature Correlation Heatmap")
plt.show()

plt.figure(figsize=(6,4))
sns.countplot(x='Cluster', hue='stabf', data=df, palette='pastel')
plt.title("Cluster Distribution by Stability")
plt.show()

# --- SPLITTING ---
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

# --- MODELLING ---
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

models = {
    "SVM": SVC(probability=True),
    "LR": LogisticRegression(),
    "RF": RandomForestClassifier(n_estimators=100),
    "XGB": XGBClassifier(eval_metric='logloss')
}

results = []
for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
    duration = time.time() - start
    results.append([name, acc, prec, rec, f1, mcc, auc, duration])

results_df = pd.DataFrame(results, columns=["Model","Acc","Prec","Rec","F1","MCC","AUC","TrainTime(min)"])
results_df["TrainTime(min)"] = results_df["TrainTime(min)"]/60
print(results_df)

# --- ROC CURVES ---
plt.figure(figsize=(8,6))
for name, model in models.items():
    fpr, tpr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    plt.plot(fpr, tpr, label=f"{name} (AUC={roc_auc_score(y_test, model.predict_proba(X_test)[:,1]):.2f})")
plt.plot([0,1],[0,1],'--',color='gray')
plt.legend()
plt.title("ROC Curves for Models (All Features)")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()

# --- SHAP ANALYSIS ---
best_model = RandomForestClassifier().fit(X_train, y_train)
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values[1], X_test, plot_type="bar")

# --- LIME EXPLANATION ---
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=np.array(X_train),
    feature_names=X.columns,
    class_names=['Stable','Unstable'],
    mode='classification'
)
exp = lime_explainer.explain_instance(X_test.iloc[5], best_model.predict_proba)
exp.show_in_notebook(show_table=True)

# --- STATISTICAL TESTING ---
anova_p = stats.f_oneway(*[model.predict(X_test) for model in models.values()])[1]
friedman_p = stats.friedmanchisquare(*[model.predict(X_test) for model in models.values()])[1]
print(f"ANOVA p-value: {anova_p:.4f}, Friedman p-value: {friedman_p:.4f}")

# --- FEATURE IMPORTANCE SUMMARY ---
importances = pd.Series(best_model.feature_importances_, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(8,5))
sns.barplot(x=importances, y=importances.index, palette="Set3")
plt.title("Feature Importance (Random Forest)")
plt.show()
